In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP'

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy==1.23.1

In [3]:
#kobert
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-kfnodmaf/kobert-tokenizer_b77e5d41fac047f19385b24bf3ab7bd9
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-kfnodmaf/kobert-tokenizer_b77e5d41fac047f19385b24bf3ab7bd9
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=74d119bd38d0a55d5f007259d11b45a5517af8d4bef2158724d2e60fb0e1cf60
  Stored in directory: /tmp/pip-ephem-wheel-cache-kz1fidda/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [5]:
#  Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

https://pleasestudy-alswldi.tistory.com/122

In [6]:
#GPU 설정
device = torch.device("cuda:0")

In [7]:
#dataset load
import pandas as pd
train_dataset = pd.read_json("df_final.json", orient="records", lines=True)

In [8]:
train_dataset.head()

,info.gender,info.ageRange,info.experience,question.raw.text,answer.summary.text,category
0,0,2,1,업무를 하다 보면 스케줄에 없던 일이 생길 때가 많이 발생합니다 그럴 때마다 지원자...,모든 게 계획대로 진행되고 예상치 못한 일이 벌어지는 것은 누구나 좋아하지 않는 상...,1
1,0,2,1,시간이 촉박한 상황에서는 시간 관리를 보통 어떻게 하시죠.,"저는 현재 처해진 상황을 정리하고, 시급성이나 중요도에 따라 우선 순위를 부여하는 ...",1
2,0,2,1,직무에서 일할 충분한 역량을 갖추고 있나요,저는 다양한 직급에서의 역할과 다양한 직무를 동시에 경험했다는 점에서 매우 어떤 프...,1
3,0,2,1,요즘 게임을 많이 합니다 어 본인께서도 게임을 즐기시 즐기시죠 만약 자신이 게임 속...,누구나 꿈꿔 왔던 가상의 공상 망상 속에 개인을 얘기한다면 전지전능하고 혼자서 모든...,1
4,0,2,1,어 본인이 개인적으로 꾸준히 스터디나 공부를 하실 텐데요 그 공부를 강의나 강의를 ...,이슈가 발견되면 그것을 직무를 시행하는 과정에서 꼭 필요한 부분으로 그것을 습득을 ...,1


In [9]:
if 'category' in train_dataset.columns:
    category_counts = train_dataset['category'].value_counts()
    print("카테고리별 개수:")
    print(category_counts)
else:
    print("데이터프레임에 'category' 열이 없습니다.")

카테고리별 개수:
category
1    5539
4    5000
3    5000
0    5000
2    5000
Name: count, dtype: int64


In [10]:
data_list = []
for q, label in zip(train_dataset['answer.summary.text'], train_dataset['category'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [22]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(f"Train dataset length: {len(dataset_train)}, Test dataset length: {len(dataset_test)}")

Train dataset length: 19154, Test dataset length: 6385


In [14]:
# dataset_train이 리스트라면 DataFrame으로 변환
if isinstance(dataset_train, list):
    dataset_train = pd.DataFrame(dataset_train)

# 이후 head() 호출
print(dataset_train.head())

                                                   0  1
0  전공 교과목 중 소프트웨어와 관련 과목들이 개설되어 있지만 자바와 관련한 프로그래밍...  0
1  이 직무에 지원한 이유는 제가 가장 업무를 잘 뽐낼 수 있는 곳이라 생각했기 때문입...  4
2  방 탈출 게임 또는 퀘스트를 깨 가는 게임 중에서 리더의 말을 따르고 전하고 팀원들...  3
3  배달이라는 것은 정말 큰 시장이구나 배달 시장이 장난 아니구나 라는 것을 느꼈고 다...  3
4  이 계열사는 블라인드, 잡 플레이닝에서 정보를 얻었고 원하는 전공과 비슷하고 나아갈...  4


In [16]:
#불용어처리
import re
import nltk

stop_words = {
    '.', '수', ',', '저는', '있는', '있습니다', '생각합니다', '때문에', '이', '그',
    '잘', '할', '것', '더', '때', '제가', '일을', '가장', '하는', '많이', '같습니다',
    '어떤', '하고', '것이', '것입니다', '다른', '것을', '있고', '대한', '많은', '좋은',
    '합니다', '한', '그래서', '될', '있다고', '위해', '가지고', '싶습니다', '굉장히',
    '되었습니다', '그런', '해', '있을', '하지만', '것은', '그리고', '큰', '있기', '제',
    '함께', '또', '했습니다'
}

# 텍스트 전처리 함수
def clean_text(text):
    if not isinstance(text, str):
        return ""

    words = text.split()  # 텍스트를 단어 단위로 나눔
    filtered_words = [word for word in words if word not in stop_words]  # 불용어 제거
    filtered_text = ' '.join(filtered_words)
    return filtered_text


# 'text' 컬럼에 대해 전처리 적용
dataset_train[0] = dataset_train[0].apply(clean_text)

In [17]:
# 이후 head() 호출
print(dataset_train.head())

                                                   0  1
0  전공 교과목 중 소프트웨어와 관련 과목들이 개설되어 있지만 자바와 관련한 프로그래밍...  0
1  직무에 지원한 이유는 업무를 뽐낼 곳이라 생각했기 때문입니다. 프로그래밍을 거의 못...  4
2  방 탈출 게임 또는 퀘스트를 깨 가는 게임 중에서 리더의 말을 따르고 전하고 팀원들...  3
3  배달이라는 정말 시장이구나 배달 시장이 장난 아니구나 라는 느꼈고 다양한 계층의 사...  3
4  계열사는 블라인드, 잡 플레이닝에서 정보를 얻었고 원하는 전공과 비슷하고 나아갈 방...  4


In [18]:
# DataFrame을 2D 리스트로 변환
dataset_train_list = dataset_train.values.tolist()

# 결과 확인
print(dataset_train_list[:5])  # 첫 5개 항목 출력

[['전공 교과목 중 소프트웨어와 관련 과목들이 개설되어 있지만 자바와 관련한 프로그래밍 수업이 개설되어 있지 않아 자바 프로그래밍을 따로 공부하고 있습니다. 교재를 구매하여 스터디원들과 결과를 비교해 보는 방식으로 스터 디를 진행하면서 도움을 얻었고 앞으로 백엔드 부서에서도 역량을 발휘할 있었으면 좋겠습니다.', '0'], ['직무에 지원한 이유는 업무를 뽐낼 곳이라 생각했기 때문입니다. 프로그래밍을 거의 못했었는데 어느 정도 레벨업을 하면서 한계를 느꼈고 문제를 해결하고 싶었는데 코딩과 수학은 곳에 취직하기 일이기는 시간을 투자하기가 답답했습니다.', '4'], ['방 탈출 게임 또는 퀘스트를 깨 가는 게임 중에서 리더의 말을 따르고 전하고 팀원들에게 영향력을 펼치는 역할을 맡아서 수행하고 싶습니다. 단체 게임을 때는 팀원들과의 합도 중요하고 함께하는 부분에 의의를 둘 거 같기 점들을 수행해 나가고 만들어 가는 과정들도 재미있고 흥미롭다고 생각하기 점을 살려서 플레이', '3'], ['배달이라는 정말 시장이구나 배달 시장이 장난 아니구나 라는 느꼈고 다양한 계층의 사람들을 만나면서 배달이 꼭 나쁜 사람만이 아니라는 깨닫게 되었습니다. 실제로 전업으로 배달업에 종사하는 종사자들도 정말 많고 그만큼 수익을 보는 사람들도 들었습니다. 돈이 많건 적건 이런 저의 취미이자 배달을 계속 조금씩 무리해 가지 않으면서 예정입니다.', '3'], ['계열사는 블라인드, 잡 플레이닝에서 정보를 얻었고 원하는 전공과 비슷하고 나아갈 방향 매출액 직원 평가들을 받았습니다. 첫 회사의 중요한 부분은 회사 분위기라고 생각합니다.', '4']]


In [23]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [24]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [25]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')


data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [28]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [29]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [30]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to('cpu')

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [31]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-31-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/300 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.210414409637451 train acc 0.078125
epoch 1 batch id 201 loss 0.480446994304657 train acc 0.5475746268656716
epoch 1 train acc 0.6592650462962962


<ipython-input-31-c43874aa7363>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/100 [00:00<?, ?it/s]

...

epoch 4 test acc 0.98234375


  0%|          | 0/300 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0077042244374752045 train acc 1.0
epoch 5 batch id 201 loss 0.0030147600919008255 train acc 0.9983675373134329
epoch 5 train acc 0.9986458333333333


  0%|          | 0/100 [00:00<?, ?it/s]

epoch 5 test acc 0.97984375


출처: https://pleasestudy-alswldi.tistory.com/122

for e in range(num_epochs): 주어진 epoch 수만큼 반복.

model.train(): 모델을 학습 모드로 설정.

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)): 학습 데이터셋을 불러오고 반복문을 통해 각 배치에 대한 학습을 진행.

optimizer.zero_grad(): 옵티마이저의 그래디언트를 초기화.
데이터를 GPU로 전송.
모델에 데이터를 입력으로 전달하여 출력을 얻음.
손실을 계산.
역전파를 수행하고 그래디언트 클리핑을 적용.
옵티마이저를 통해 파라미터를 업데이트
스케줄러를 통해 학습률을 업데이트.
정확도 계산.
현재의 학습 상태를 출력.( 일정 간격마다  )
모델 평가부분은 다음과 같다.

model.eval(): 모델을 평가 모드로 변경.

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)): 테스트 데이터셋을 불러오고 반복문을 통해 각 배치에 대한 평가를 진행.

In [ ]:
# info.gender -> MALE=0, FEMALE=1
# info.ageRange -> ['-34', '35-44', '55-', '45-54'] -> 0, 1, 2, 3
# info.experience -> ['NEW', 'EXPERIENCED'] -> 0, 1
# category_range_map = {'\'technology\'': 0, '\'attitude\'': 1, '\'background\'': 2, '\'personality\'': 3, '\'etc\'': 4}

In [35]:
# 데이터셋 로드
train_dataset = pd.read_json("df_final.json", orient="records", lines=True)

# 데이터셋에서 랜덤으로 질문과 카테고리 가져오기
def get_random_question_and_category(dataset):
    # 랜덤으로 데이터에서 질문과 카테고리 선택
    random_row = dataset.sample(n=1)  # 랜덤 샘플 1개 선택
    question = random_row['question.raw.text'].values[0]  # 질문
    category = random_row['category'].values[0]  # 카테고리
    return question, category

def predict(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()  # 모델 평가 모드로 설정

    # 예측 수행
    with torch.no_grad():  # 평가 중에는 그래디언트 계산을 하지 않음
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)

            valid_length = valid_length
            label = label.long().to(device)

            out = model(token_ids, valid_length, segment_ids)

            test_eval = []
            for i in out:
                logits = i
                logits = logits.detach().cpu().numpy()

                # 라벨 매핑
                if np.argmax(logits) == 0:
                    test_eval.append("기술적(0)")
                elif np.argmax(logits) == 1:
                    test_eval.append("태도와 관련(1)")
                elif np.argmax(logits) == 2:
                    test_eval.append("배경과 관련(2)")
                elif np.argmax(logits) == 3:
                    test_eval.append("성격과 관련(3)")
                elif np.argmax(logits) == 4:
                    test_eval.append("기타(4)")

            return test_eval[0]  # 예측된 카테고리 반환

# 질문-답변 시스템
def question_answer_system(dataset):
    end = 1
    while end == 1:
        # 랜덤 질문과 실제 카테고리 가져오기
        question, true_category = get_random_question_and_category(dataset)

        print(f"질문: {question}")

        # 사용자로부터 답변 받기
        sentence = input("답변을 입력해주세요 (종료하려면 0 입력): ")

        # 종료 조건
        if sentence == "0":
            break

        # 모델을 통해 카테고리 예측
        predicted_category = predict(sentence)

        # 예측된 카테고리와 실제 카테고리 출력
        print(f"모델의 예측 카테고리: {predicted_category}")
        print(f"실제 카테고리: {'기술적(0)' if true_category == 0 else '태도와 관련(1)' if true_category == 1 else '배경과 관련(2)' if true_category == 2 else '성격과 관련(3)' if true_category == 3 else '기타(4)'}")
        print("\n")

# 질문과 답변 시스템 실행
question_answer_system(train_dataset)

질문: 본인은 해당 분야에 지원하기 위해 어떠한 노력을 하셨나요
답변을 입력해주세요 (종료하려면 0 입력): 정보처리기사와 대외활동을 했습니다.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


모델의 예측 카테고리: 배경과 관련(2)
실제 카테고리: 배경과 관련(2)


질문: 대학에서 배운 지원자님의 전공 과목 중 가장 기억에 남는 과목을 한 가지를 골라 설명해 주세요
답변을 입력해주세요 (종료하려면 0 입력): 자연언어처리입니다! 왜냐면 어렵기 때문입니다!
모델의 예측 카테고리: 태도와 관련(1)
실제 카테고리: 기술적(0)


질문: 지원자님께서는 사람들과 협업을 하며 가장 어려웠던 점이 있다면 무엇이었나요 또 그것을 어떻게 극복했는지도 궁금합니다 말씀해 주세요
답변을 입력해주세요 (종료하려면 0 입력): it 공부를 했습니다.
모델의 예측 카테고리: 태도와 관련(1)
실제 카테고리: 성격과 관련(3)


질문: 지원자님께서는 해당 분야에 지원하기 위해서 관련된 어떤 노력들을 하셨는지 궁금한데요 노력하셨던 것들을 모두 설명해 주시면 감사하겠습니다
답변을 입력해주세요 (종료하려면 0 입력): 모르겠습니다.
모델의 예측 카테고리: 태도와 관련(1)
실제 카테고리: 기타(4)


질문: 일을 하다 보면 업무량이 과도하게 많아질 수도 있는데 그래도 잘 버틸 수 있는 자신이 있을까요
답변을 입력해주세요 (종료하려면 0 입력): 네! 할 수 있습니다!
모델의 예측 카테고리: 태도와 관련(1)
실제 카테고리: 태도와 관련(1)


질문: 지원자님께서 본인이 현재 가지고 있는 위치보다 어려운 목표를 달성한 적이 있으신가요 있다면 해당 경험에 대해서 구체적으로 말씀 부탁드립니다
답변을 입력해주세요 (종료하려면 0 입력): 네 저는 it 역량을 준비했습니다.
모델의 예측 카테고리: 태도와 관련(1)
실제 카테고리: 성격과 관련(3)


질문: 대학 생활을 보내면서 가장 힘들었던 경험은 무엇인가요 또한 지원자님은 그 힘든 경험을 어떻게 극복하셨었는지도 말씀해 주시기 바랍니다
답변을 입력해주세요 (종료하려면 0 입력): 자격을 준비했씁니다.
모델의 예측 카테고리: 태도와 관련(1)
실제 카테고리: 배경과 관련(2)


질문: 지원자에게 질문 하나 해 보겠습니다 만약

KeyboardInterrupt: Interrupted by user

In [ ]:
#https://pleasestudy-alswldi.tistory.com/164 모델 저장 및 사용

Macro-F1 Score
둘 이상의 클래스가 있는 데이터 세트의 전체 F1 점수를 계산하기 위한 집계 방식 중 하나이다.

In [39]:
# 이미 선언한 train_dataloader와 test_dataloader를 사용합니다.
# 예시:
# train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
# test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# 모델 평가 함수
def evaluate_model(model, test_loader):
    model.eval()  # 모델을 평가 모드로 전환
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, labels) in enumerate(test_loader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            labels = labels.long().to(device)

            # 모델의 출력값 계산
            outputs = model(token_ids, valid_length, segment_ids)

            # 예측값 계산
            _, predicted = torch.max(outputs, 1)

            true_labels.extend(labels.cpu().numpy())  # 실제 값
            predicted_labels.extend(predicted.cpu().numpy())  # 예측된 값

    # F1 Score 계산 (전체 클래스에 대해)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')  # weighted F1 score
    macro_f1 = f1_score(true_labels, predicted_labels, average='macro')  # Macro F1 score

    return f1, macro_f1

# 테스트 데이터셋에 대해 모델 평가
f1, macro_f1 = evaluate_model(model, test_dataloader)

print(f"F1 Score (Weighted): {f1:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")


F1 Score (Weighted): 0.9795
Macro F1 Score: 0.9799


In [40]:
path = '/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP/models/'
torch.save(model, path + 'sentiment_analysis_model_epoch10.pt')  # 전체 모델 저장

torch.save(model.state_dict(), 'sentiment_analysis_model_state_dict_epoch10.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'sentiment_analysis_model_all_epoch10.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

KeyboardInterrupt: 

In [43]:
!git add df_final.json
!git commit -m "add data_final.json"

[main bb16894] add data_final.json
 1 file changed, 25539 insertions(+)
 create mode 100644 df_final.json
